<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/MagicSquares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Magic Square Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

A magic square is an arrangement of distinct integers in a square grid, such that the values in each row, in each
column and in the two main diagonals all add up to the same number. If n denotes the number of cells,
the values 1 to n are to be distributed.

In [1]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobu

In [2]:
from ortools.sat.python import cp_model
from itertools import product
import math

Define magic square size

Create model

In [21]:
n = 4*4
size_float = math.sqrt(n)
size = int(size_float)
if size != size_float:
  raise ValueError(f'sqrt(n) not working for n = {n}')

model = cp_model.CpModel()
# Type your model here ...
board = [[model.NewIntVar(1, n, f"({i},{j})") for j in range(size)] for i in range(size)]

model.AddAllDifferent([cell for row in board for cell in row])

# my own code
# for i in range(1, size):
#   model.Add(sum([board[i][j] for j in range(size)]) == sum(board[0][j] for j in range(size))) # Rows should have same sum as row 0
#   model.Add(sum([board[j][i] for j in range(size)]) == sum(board[j][0] for j in range(size))) # Columns should have same sum as column 0

# model.Add(sum(board[0][j] for j in range(size)) == sum(board[j][0] for j in range(size))) # Columns and rows should have the same sum
# model.Add(sum([board[i][i] for i in range(size)]) == sum(board[0][j] for j in range(size))) # diagonal 1
# model.Add(sum([board[i][size-1-i] for i in range(size)]) == sum(board[0][j] for j in range(size))) # diagonal 2


# model.Add(board[0][0] == 9)
# model.Add(board[0][size-1] == 8)
# model.Add(board[size-1][0] == 6)
# model.Add(board[size-1][size-1] == 11)


# optimized slightly:
model = cp_model.CpModel()
board = [[model.NewIntVar(1, n, f"({i},{j})") for j in range(size)] for i in range(size)]

model.AddAllDifferent([cell for row in board for cell in row])

# Store the sum of the first row and column in a variable
first_row_sum = sum(board[0][j] for j in range(size))
first_col_sum = sum(board[j][0] for j in range(size))

for i in range(1, size):
  model.Add(sum(board[i][j] for j in range(size)) == first_row_sum) # Rows should have same sum as row 0
  model.Add(sum(board[j][i] for j in range(size)) == first_col_sum) # Columns should have same sum as column 0

model.Add(first_row_sum == first_col_sum) # Columns and rows should have the same sum

# Use a single loop for the diagonal constraints
diag1_sum = 0
diag2_sum = 0
for i in range(size):
    diag1_sum += board[i][i]
    diag2_sum += board[i][size-1-i]
model.Add(diag1_sum == first_row_sum) # diagonal 1
model.Add(diag2_sum == first_row_sum) # diagonal 2

model.Add(board[0][0] == 9)
model.Add(board[0][size-1] == 8)
model.Add(board[size-1][0] == 6)
model.Add(board[size-1][size-1] == 11)


# further optimizations
# Symmetry breaking constraint
model.Add(board[0][1] < board[0][size-1])

# Decision strategy
vars = [cell for row in board for cell in row]
model.AddDecisionStrategy(vars, cp_model.CHOOSE_FIRST, cp_model.SELECT_MIN_VALUE)

# another symmetry breaking constraint, this time in the col
model.Add(board[1][0] < board[size-1][0])

Callback for solution printing (adapt if you do not use an n*n board)

In [4]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables

    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{self.Value(self.__variables[i][j])}] ", end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [22]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(board))
print(status)

[9] [1] [16] [8] 

[4] [12] [13] [5] 

[15] [7] [2] [10] 

[6] [14] [3] [11] 




4


In [23]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")

# previous code
# Runtime:   0.223372703ms
# Booleans:  252
# Failures:  494
# Branches:  3932

# optimized version
# Runtime:   0.174512787ms
# Booleans:  252
# Failures:  494
# Branches:  3932

# one more symmetry breaking constraint
# Runtime:   0.11152139200000001ms
# Booleans:  228
# Failures:  128
# Branches:  3012

# added Decision strategy
# Runtime:   0.09832262400000001ms
# Booleans:  228
# Failures:  128
# Branches:  3012

# another symmetry breaking constraint added
# Runtime:   0.039061308ms
# Booleans:  180
# Failures:  59
# Branches:  2042

Runtime:   0.039061308ms
Booleans:  180
Failures:  59
Branches:  2042
